# Constituer un corpus de critiques de films

## Interroger les pages Web de chaque film

**Objectif :** lire le fichier `liste_liens.txt` afin d’extraire les identifiants des films et interroger les pages Web concernées

**Besoins :**
- ouvrir le fichier en lecture et en récupérer le contenu
- parcourir chaque ligne
- analyser la syntaxe de la ligne
- isoler l’identifiant
- reconstruire les urls sur *Allociné*


**Rappel :** identifiants à la fin de chaque URL, composés uniquement de chiffres
```txt
/film/fichefilm_gen_cfilm=114782.html
/film/fichefilm_gen_cfilm=143692.html
```

**1e étape :** ouvrir le fichier en mode lecture et en récupérer le contenu

In [ ]:
# Ouvrir fichier en lecture
with open('liste_liens.txt') as file:
    # Récupérer les lignes
    lines = file.readlines()

**2e étape :** parcourir chaque ligne

In [ ]:
# Pour chaque ligne dans la liste des lignes
for line in lines:
    # Analyser sa syntaxe
    pass

**3e étape :** isoler l’identifiant du film dans la ligne

- importer le module des expressions régulières

In [ ]:
import re

- méthode `search()` du module `re` pour exécuter *regex*

In [ ]:
for line in lines:
    # Rechercher un motif dans la ligne
    id_movie = re.search(pattern, line)

- motif pour chiffres qui se suivent : `[0-9]+` ou `\d+`

In [ ]:
for line in lines:
    # Rechercher un motif dans la ligne
    id_movie = re.search('\d+', line)

- résultat de la capture disponible via méthode `group()` :

In [ ]:
for line in lines:
    id_movie = re.search('\d+', line)
    # Affichage de l'identifiant de chaque film
    print(id_movie.group(0))

**4e étape :** reconstruire les URL sur Allociné

- critiques spectateurs sur une page où `{id}` est l’identifiant du film :
```txt
http://www.allocine.fr/film/fichefilm-{id}/critiques/spectateurs/
```

In [ ]:
for line in lines:
    id_movie = re.search('\d+', line)
    # Constuction de l'URL
    url = 'http://www.allocine.fr/film/fichefilm-{id}/critiques/spectateurs/'

- méthode `format()` sur une chaîne de caractères pour remplacer `{id}` de l’URL par l’identifiant du film :

In [ ]:
for line in lines:
    id_movie = re.search('\d+', line)
    url = 'http://www.allocine.fr/film/fichefilm-{id}/critiques/spectateurs/'
    # Formatage de chaque URL
    url = url.format(id = id_movie.group(0))

- ou, mieux, utiliser une *f-string* :

In [ ]:
for line in lines:
    id_movie = re.search('\d+', line)
    url = f'http://www.allocine.fr/film/fichefilm-{id_movie.group(0)}/critiques/spectateurs/'

**5e étape :** importer le fichier `tools.py` du répertoire `modules` et appeler la méthode `get_HTML_from_URL()` pour récupérer le contenu HTML de chaque page

In [ ]:
import re, modules.tools as tools

In [ ]:
for line in lines:
    id_movie = re.search('\d+', line)
    url = f'http://www.allocine.fr/film/fichefilm-{id_movie.group(0)}/critiques/spectateurs/'
    # Récupérer code HTML
    html = tools.get_HTML_from_URL(url)

## Extraire les critiques

Sur les pages de chaque film, les critiques utilisateur sont encadrées de marqueurs auxquels sont appliqués la classe CSS `content-txt`

Grâce à *BeautifulSoup*, on peut facilement sélectionner ces marqueurs et en récupérer le contenu

In [ ]:
from bs4 import BeautifulSoup

for line in lines:
    id_movie = re.search('\d+', line)
    url = f'http://www.allocine.fr/film/fichefilm-{id_movie.group(0)}/critiques/spectateurs/'
    html = tools.get_HTML_from_URL(url)
    # Extraire les critiques utilisateur
    soup = BeautifulSoup(html, 'html.parser')
    reviews = soup.select('.content-txt')

Comme il s’agit d’une procédure déjà utilisée dans le précédent *notebook*, autant définir une fonction à placer dans le module *tools* :

In [ ]:
from bs4 import BeautifulSoup

def parse_HTML_by_class(html, selector):
    """
        Extrait des balises d’une page HTML grâce à un sélecteur CSS.
    """
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.select(selector)
    return tags

Et modifier le code en conséquence :

In [ ]:
for line in lines:
    id_movie = re.search('\d+', line)
    url = f'http://www.allocine.fr/film/fichefilm-{id_movie.group(0)}/critiques/spectateurs/'
    html = tools.get_HTML_from_URL(url)
    # Extraire les critiques utilisateur
    reviews = tools.parse_HTML_by_class(html, '.content-txt')

Pour placer au final toutes les critiques dans une liste :

In [ ]:
movie_reviews = []

for line in lines:
    id_movie = re.search('\d+', line)
    url = f'http://www.allocine.fr/film/fichefilm-{id_movie.group(0)}/critiques/spectateurs/'
    html = tools.get_HTML_from_URL(url)
    # Ajouter les critiques à la liste complète
    movie_reviews.append(
        # Chaque enregistrement = id film, liste des critiques associées
        (id_movie.group(0), tools.parse_HTML_by_class(html, '.content-txt'))
    )

## Constituer le corpus

Maintenant que nous disposons d’une liste des critiques pour chaque film, nous souhaitons au final constituer un corpus avec les caractéristiques suivantes :
- un fichier par film
- une critique par ligne

Dans les critiques extraites par *BeautifulSoup*, nous souhaitons déjà nous débarrasser des balises HTML :

In [ ]:
# Pour chaque corpus de critiques de film…
for movie_review in movie_reviews:
    # … analyser uniquement les critiques
    reviews = movie_review[1]
    # Et pour chacune de ces critiques…
    for review in reviews:
        # … ne conserver que le contenu textuel
        review = review.get_text()

En analysant le retour grâce à la fonction `print()`, on observe que :
- les retours à la ligne utilisateur sont préservés
- il subsiste des espaces superflus avant et après chaque critique

Python fournit des méthodes sur les chaînes de caractères pour gérer ces effets :
- méthode `strip()` pour retirer les espaces
- méthode `replace()` pour retirer les retours à la ligne (`'\n'`)

In [ ]:
for movie_review in movie_reviews:
    reviews = movie_review[1]
    for review in reviews:
        review = review.get_text()
        # Supprimer les espaces au début et à la fin de chaque critique
        review = review.strip()
        # Remplacer les retours à la ligne par des simples espaces
        review = review.replace('\n', ' ')

Mieux, il est possible de chaîner les méthodes :

In [ ]:
for movie_review in movie_reviews:
    reviews = movie_review[1]
    for review in reviews:
        review = review.get_text().strip().replace('\n', ' ')

Il ne reste plus qu’à enregistrer les critiques :

In [ ]:
for movie_review in movie_reviews:
    identifier = movie_review[0]
    reviews = movie_review[1]
    for review in reviews:
        review = review.get_text().strip().replace('\n', ' ')
        with open(f'reviews_{identifier}.txt', 'a') as file:
            # Ecriture de la ligne
            file.write(review)
            file.write('\n')